<h1><center>  文档摘要 -  Nucleus API实例</center></h1>


<h1><center>  所有权及保密条款属SumUp Analytics所有 </center></h1>


<h1><center>  免责声明和服务条款可通过 www.sumup.ai 获取 </center></h1>


 


## 目标: 
-	开发一个流程来定制和微调文档摘要


## 数据:
-	来自同一领域的任何文档集合，可以在文档类型等分类方面进一步细化
    **Nucleus Datafeed可提取主要中央银行的所有内容**


## Nucleus APIs:
-	数据集创建 API
 - 	*api_instance.post_upload_file(file, dataset)*
 - 	*nucleus_helper.import_files(api_instance, dataset, file_iters, processes=1)*

        nucleus_helper.import_files利用api_instance.post_upload_file并行执行来加速数据集的创建


-	主题建模 API
 - 	*api_instance.post_topic_api(payload)*


-	主题情绪 API
 - 	*api_instance.post_doc_summary_api(payload)*



## 方法:

### 1.	数据集准备
-	创建一个Nucleus数据集，其中包含所选历史时期内的所有相关文档

    

In [ ]:
import csv
import json
import nucleus_api.api.nucleus_api as nucleus_helper
import nucleus_api
from nucleus_api.rest import ApiException

configuration = nucleus_api.Configuration()
configuration.host = 'UPDATE-WITH-API-SERVER-HOSTNAME'
configuration.api_key['x-api-key'] = 'UPDATE-WITH-API-KEY'

# 创建API实例
api_instance = nucleus_api.NucleusApi(nucleus_api.ApiClient(configuration))

In [ ]:
print('---- Case 1: you are using your own corpus, coming from a local folder ----')
folder = 'Corporate_documents'         
dataset = 'Corporate_docs'# str | 文件的目标数据集。

# 以递归方式从文件夹构建文件。 
# 每项都采用以下格式
# {'filename': filename,   # 要上传的文件名。 必填
#  'metadata': {           # 该文件的元数据。 可选
#      'key1': val1,       # 只要名称，密钥就可以有任意名称
#      'key2': val2        # 包含字母数字（0-9 | a-z | A-Z）和下划线（_）
#   } 
# }
file_iter = []
for root, dirs, files in os.walk(folder):
    for file in files:
        #if Path(file).suffix == '.pdf': # .txt .doc .docx .rtf .html .csv also supported
            file_dict = {'filename': os.path.join(root, file),
                         'metadata': {'company': 'Apple',
                                      'category': 'Press Release',
                                      'date': '2019-01-01'}}
            file_iter.append(file_dict)

file_props = nucleus_helper.upload_files(api_instance, dataset, file_iter, processes=4)
for fp in file_props:
    print(fp.filename, '(', fp.size, 'bytes) has been added to dataset', dataset)

    
    
print('---- Case 2: you are using an embedded datafeed ----')
dataset = 'sumup/central_banks_chinese'# 从Nucleus自带数据。
metadata_selection = {'bank': 'people_bank_of_china', 'document_category': ('speech', 'press release', 'publication')}

### 2.	文件摘要
-	使用以下输入参数来控制摘要的大小并过滤过短或过长的句子。
    - `summary_length`
    - `context_amount` (每个关键摘要句子周围的句子数)
    - `short_sentence_length`
    - `long_sentence_length`


-	创建自定义停用词列表（`custom_stop_words`）以调整或指定摘要的内容类型
    - `custom_stop_words` (自定义停用词列表)


-	接下来，我们将讨论如何构建自定义的停用词列表以优化文档摘要



In [ ]:
print('---------------- Get doc summaries ------------------------')
# 这些都是摘要API的可能输入参数
custom_stop_words = ["decree","motion"] # 停用词列表。 （可选）
summary_length = 6 # int | 用户希望在文档摘要中看到的最多点句数。 （可选）（默认为6）
context_amount = 0 # int | 围绕关键摘要句子的句子数。 （可选）（默认为0）
short_sentence_length = 0 # int | 从摘要中排除的短句子长度（可选）（默认为4）
long_sentence_length = 40 # int | 从摘要中排除的长句子长度（可选）（默认为40）
payload = nucleus_api.DocumentSummaryModel(dataset='Corporate_docs', 
                                        doc_title='my_title', 
                                        summary_length=summary_length)
try:
    api_response = api_instance.post_doc_summary_api(payload)
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:   
    print('Summary for', api_response.result.doc_title)
    for sent in api_response.result.summary.sentences:
        print('    *', sent)


### 3.	微调

#### a.	提取在语料库文档中找到的主题

-   通过使用Doc Summary API中的`custom_stop_words`参数排除次要或无关的主题以定制对比度分析


-	提取具有相似属性（即文档类型，内容）的文档子集的关键主题，并打印这些主题的关键字

In [ ]:
print('------------- Get list of topics from dataset --------------')

payload = nucleus_api.Topics(dataset='Corporate_docs',                         
                            query='',                       
                            num_topics=8, 
                            num_keywords=8,
                            metadata_selection=metadata_selection)
try:
    api_response = api_instance.post_topic_api(payload)        
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:       
    for i, res in enumerate(api_response.result.topics):
        print('Topic', i, ' keywords: ', res.keywords)    
        print('---------------')

使用您的域专业知识或客户/顾问输入，您可以确定特定主题或关键字是否没有足够的区别来为文档摘要做出贡献。

然后，您可以通过创建包含这些单词的`custom_stop_words`变量来定制文档对比摘要。 如下所示初始化变量并将其传递到第2节主代码的有效负载中：

In [1]:
custom_stop_words = ["decree","motion"] # str | 停用词列表。 （可选）

#### b.	分离语库中特定的文档子集

**Document types**: 您可以通过利用构建数据集期间提供的元数据选择器来优化主题的提取和隔离非主题/次要主题，以获得您感兴趣的任何级别的粒度。

创建一个变量`metadata_selection`并将其传递给有效负载。 在整个语料库的子集上重新运行`post_topic_api`。
 

In [ ]:
# 如果您创建了一个数据集，其中一个元数据是文档的类别，
# 这个类别的一个值可能是'演讲'
# 您可以专注于主题分析，并为语库中的所有语音文档创建自定义的停用词列表，以便在稍后的生产部署中用
metadata_selection = {"document_category": "speech"}   # str | json object of {\"metadata_field\":[\"selected_values\"]} (optional)

#### c. 在语料库中的某些主题上创建自定义停用词列表

**query**: 您可以通过利用Topic API的查询变量来优化主题的提取和隔离非主题/次要主题。

创建一个变量`query`并将其传递给有效负载。 在你的语库上重新运行`post_topic_api`，提到一个或多个特定单词。

In [ ]:
query = '(veto rights OR jury decision OR verdict)' # str | 全文查询，使用mysql MATCH boolean query格式。 示例：“（word1 OR word2）AND（word3 OR word4）”（可选）

版权(c) 2019年 SumUp Analytics 公司 版权所有。 

通知：所有信息均属于SumUp Analytics Inc公司及其供应商的财产。 本合同所包含的知识产权和技术概念属于SumUp Analytics Inc.及其供应商的专利，可由美国和外国专利、在工艺中的专利以及受贸易秘密或版权法保护的专利涵盖。 

除非得到SumUp Analytics公司的事先书面批准，否则严禁传播此类信息或复制此材料。 